**ÉTAPE 0** : préparation des données

In [1]:
import process 
import pandas as pd

# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
# Jerome path : 'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DATA_Statapp.csv'
# Mohamed path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

# Apply conversion function to 'open' and 'close' columns
df['open'] = df['open'].apply(process.safe_literal_eval)
df['close'] = df['close'].apply(process.safe_literal_eval)

# Calculate returns for each line
df['return'] = df.apply(lambda row: [(close - open) / open for open, close in zip(row['open'], row['close'])], axis=1)

new_df = df[['ticker', 'return']] # create a new data frame with the column ticker and return 

# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

df_cleaned = process.remove_rows_with_nan(returns_df)
df_cleaned.reset_index(drop=True, inplace=True)

process.check_nan_inf(df_cleaned)

df_cleaned.shape

There are no NaN values in the dataframe


(632, 5532)

**ÉTAPE 1** : Phase d'entraînement

1. Obtention de la matrice de corrélation des actifs sur une fenêtre arrière de 30 jours (1 mois)

In [2]:
lookback_window = 30
correlation_matrix = process.correlation_matrix(lookback_window, df_cleaned)

2. Obtention de la composition de chaque cluster et du centroïde de chacun d'entre eux

In [3]:
## PROBLÈME DES ARRONDIS

cluster_composition = process.cluster_composition_and_centroid(df_cleaned=df_cleaned, correlation_matrix=correlation_matrix, number_of_clusters=20, lookback_window=30)

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


**ÉTAPE 2** : construction de portefeuille

1. On donne, au sein d'un même cluster, un poids à chaque actif selon sa distance au centroïde de celui-ci. Cela nous servira plus tard pour calculer le rendement de chaque cluster (alors vu comme un nouvel actif synthétique)

In [4]:
constituent_weights = process.constituent_weights(df_cleaned=df_cleaned, cluster_composition=cluster_composition, sigma=10, lookback_window=30)

In [53]:
## poids très proches ... ==> dû au fait qu'on regarde sur un trop petit échantillon (30 jours) ? 

constituent_weights


[['cluster 1',
  [['AFL', 0.971927270260057],
   ['AXP', 0.9715114293769647],
   ['BMO', 0.9708630734985004],
   ['BTO', 0.9710264677451717],
   ['CNI', 0.9720606662883962],
   ['DHR', 0.9718262550421967],
   ['ETR', 0.97127389032118],
   ['HON', 0.9721102874091653],
   ['HUM', 0.9738289774547463],
   ['JNJ', 0.9712460251410782],
   ['KR', 0.9711854430123082],
   ['LLY', 0.9711458049610618],
   ['LMT', 0.9711115167123083],
   ['LNC', 0.9714088943211611],
   ['MIY', 0.9699866842088706],
   ['MKC', 0.9715405412400014],
   ['MRK', 0.9718164521447056],
   ['MTG', 0.9711327431834283],
   ['NFG', 0.9704839319106298],
   ['PFE', 0.9720862111104335],
   ['PNC', 0.9721873027438807],
   ['RYN', 0.9712956172730897],
   ['SCS', 0.9726358545375253],
   ['SUP', 0.9711465380567241],
   ['SYY', 0.9718764736986515],
   ['TD', 0.9710221803714257],
   ['TMO', 0.9709896725748498],
   ['WFC', 0.9720256067661093],
   ['WMK', 0.9712988618842758],
   ['XLP', 0.971214142528056]]],
 ['cluster 2',
  [['ABT', 0.9

Le choix des rendements attendus (expected_returns) dans le modèle de Markowitz peut être un défi car il nécessite des prévisions pour chaque actif inclus dans le portefeuille. 

In [ ]:
cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=30) 

markowitz_weight = process.makowitz_weights(cluster_return=cluster_return, lookback_window=30)

W = process.final_weights(markowitz_weights=markowitz_weight, constituent_weights=constituent_weights)

In [23]:
## on récupère le dataframe contenant les return de chaque cluster

cluster_return = process.cluster_return(constituent_weights=constituent_weights, df_cleaned=df_cleaned, lookback_window=30) 

## on construit la matrice de corrélation associée à ces returns, c'est donc une matrice de corrélation de return de cluster

cov_matrix = cluster_return.cov()

## on construit le vecteur d'expected return du cluster (250 jours de trading par an, on passe de rendements journaliers à rendements annualisés)
expected_returns = (cluster_return.mean(axis=0) + 1)**(250/lookback_window) - 1 ## on fait ici le choix de prendre le rendement moyen comme objectif

In [38]:
from pypfopt.efficient_frontier import EfficientFrontier

ef = EfficientFrontier(expected_returns=expected_returns, cov_matrix=cov_matrix)
ef.max_sharpe()
weights = ef.clean_weights()

In [52]:
### On cherche désormais à calculer le poids de chaque actif dans le portefeuille total

W = []

for i in range(len(constituent_weights)):

    markowitz_weight = weights[constituent_weights[i][0]]

    for elem in constituent_weights[i][1]:

        W.append([elem[0], elem[1] * markowitz_weight])